HOSPITAL PATIENT DETERIORATION PREDICTION
Scenario: ICU Patient Health Risk Forecasting
A hospital ICU continuously monitors critical patient vitals.
The goal is to predict the patient’s risk score for the next 6 hours using the previous 12 hours of multivariate time-series data, so doctors can intervene before a medical emergency occurs.

RNN

In [65]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [66]:
lookback = 12
horizon = 6

df = pd.read_csv('vitals_dataset_48h_with_risk.csv')
features = ["HR", "BP", "O2", "Resp", "Temp"]
target = 'risk_score'
print(df.head())
print(len(df))

   Hour  HR   BP  O2  Resp  Temp  risk_score
0     1  82  120  98    16  36.8    0.000000
1     2  85  118  97    17  36.9    6.504384
2     3  88  115  96    18  37.0   13.580608
3     4  92  112  95    19  37.1   21.454831
4     5  96  108  94    20  37.2   29.900894
48


In [67]:
def make_window(df,features,target,lookback=12,horizon=6):
    Xv= df[features].values
    yv= df[target].values

    x_list,y_list = [],[]

    for i in range(len(df) - lookback - horizon +1):
        x_list.append(Xv[i:i+lookback])
        y_list.append(yv[i+lookback:i+lookback+horizon])
    return np.array(x_list),np.array(y_list)

In [68]:
X,y = make_window(df,features,target,lookback,horizon)
n = len(X)
split = int(0.8*n)

X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

In [69]:
x_scaler = StandardScaler()
N, T, F = X_train.shape

X_train_2d = X_train.reshape(-1, F)
X_val_2d   = X_val.reshape(-1, F)

x_scaler.fit(X_train_2d)
X_train = x_scaler.transform(X_train_2d).reshape(N, T, F)
X_val   = x_scaler.transform(X_val_2d).reshape(X_val.shape[0], T, F)


y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1)).reshape(y_train.shape)
y_val_scaled   = y_scaler.transform(y_val.reshape(-1, 1)).reshape(y_val.shape)


In [70]:
def build_model(input_shape, horizon):
    model = Sequential([
        SimpleRNN(64, activation="tanh", input_shape=input_shape),
        Dropout(0.25),
        Dense(32, activation="relu"),
        Dense(horizon)
    ])
    return model


In [71]:

def train_and_report(optimizer, label=""):
    tf.random.set_seed(1)

    model = build_model((lookback, F), horizon)

    # Huber loss reduces sensitivity to outliers/spikes
    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.Huber(delta=1.0),
        metrics=["mae"]
    )

    es = EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)

    history = model.fit(
        X_train, y_train_scaled,
        validation_data=(X_val, y_val_scaled),
        epochs=200,
        batch_size=8,
        callbacks=[es],
        verbose=0
    )

    best_epoch = int(np.argmin(history.history["val_loss"]))
    best_train_loss = float(history.history["loss"][best_epoch])
    best_val_loss   = float(history.history["val_loss"][best_epoch])

    if label == "":
        print(f"\nFinal train loss: {best_train_loss}")
        print(f"Final val loss  : {best_val_loss}")
    else:
        print(f"\nFinal train loss ({label}): {best_train_loss}")
        print(f"Final val loss  ({label}): {best_val_loss}")

    return model

In [ ]:

sgd  = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9, clipnorm=1.0)
adam = tf.keras.optimizers.Adam(learning_rate=0.1, clipnorm=1.0)

model_sgd  = train_and_report(sgd,  label="SGD")
model_adam = train_and_report(adam, label="Adam")

last_12_vitals = df[features].tail(lookback).values
last_12_scaled = x_scaler.transform(last_12_vitals).reshape(1, lookback, F)

pred_sgd_scaled  = model_sgd.predict(last_12_scaled,  verbose=0)[0] 
pred_adam_scaled = model_adam.predict(last_12_scaled, verbose=0)[0] 

# invert scaling to original risk_score scale
pred_sgd  = y_scaler.inverse_transform(pred_sgd_scaled.reshape(-1, 1)).reshape(-1)
pred_adam = y_scaler.inverse_transform(pred_adam_scaled.reshape(-1, 1)).reshape(-1)

last_12_risk = df[target].tail(lookback).values

print("\nLast 12 hours risk_score:", np.round(last_12_risk, 2))
print("Predicted risk next 6 hours (SGD) : ", np.round(pred_sgd, 2))
print("Predicted risk next 6 hours (Adam): ", np.round(pred_adam, 2))


d:\Training\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Final train loss (SGD): 0.0582413524389267
Final val loss  (SGD): 0.008881419897079468

Final train loss (Adam): 0.38756808638572693
Final val loss  (Adam): 0.012296125292778015

Last 12 hours risk_score: [25.91 27.54 28.48 29.38 30.77 31.89 32.59 30.1  29.2  26.2  25.97 26.41]
Predicted risk next 6 hours (SGD) :  [30.44 31.99 27.89 30.76 31.32 26.78]
Predicted risk next 6 hours (Adam):  [31.89 28.34 31.48 28.73 31.08 28.15]


Scenario: Smart Factory Temperature Monitoring
A factory monitors the daily average temperature (°C) of a critical machine.
The goal is to predict tomorrow’s temperature using the last 5 days of temperature readings in order to:
Detect overheating trends
Schedule preventive maintenance
Avoid machine failure
🔹 Sample Dataset (Time-Series)
 
Day	Temperature (°C)
Day 1	68
Day 2	70
Day 3	71
Day 4	73
Day 5	74
Day 6	76
Day 7	78
Day 8	80
Day 9	82
Day 10	85

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

tf.random.set_seed(1)
np.random.seed(1)


temps = np.array([33,51,63,55,67,88,84,90,23,84,89], dtype=np.float32)


window_size = 5
X_list, y_list = [], []
for i in range(len(temps) - window_size):
    X_list.append(temps[i:i+window_size])     
    y_list.append(temps[i+window_size])      

X = np.array(X_list, dtype=np.float32)      
y = np.array(y_list,  dtype=np.float32)       

X = X[..., np.newaxis]  
y = y[..., np.newaxis]  

print("Number of samples:", X.shape[0]) 


train_size = max(1, int(0.8 * len(X)))  
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]


x_mean, x_std = X_train.mean(), X_train.std() + 1e-8
y_mean, y_std = y_train.mean(), y_train.std() + 1e-8

X_train_n = (X_train - x_mean) / x_std
X_val_n   = (X_val   - x_mean) / x_std
y_train_n = (y_train - y_mean) / y_std
y_val_n   = (y_val   - y_mean) / y_std


def build_model(units=16):
    model = Sequential([
        SimpleRNN(units, activation='tanh', input_shape=(window_size, 1)),
        Dense(1) 
    ])
    return model


sgd_opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.5, clipnorm=1.0)

model_sgd = build_model(16)
model_sgd.compile(optimizer=sgd_opt, loss='mse', metrics=['mae'])

history_sgd = model_sgd.fit(
    X_train_n, y_train_n,
    epochs=250,          
    batch_size=5,
    validation_data=(X_val_n, y_val_n),
    verbose=0
)

print("Final train loss (SGD):", history_sgd.history['loss'][-1])
print("Final val   loss (SGD):", history_sgd.history['val_loss'][-1])


adam_opt = tf.keras.optimizers.Adam(learning_rate=0.01, clipnorm=1.0)

model_adam = build_model(16)
model_adam.compile(optimizer=adam_opt, loss='mse', metrics=['mae'])

history_adam = model_adam.fit(
    X_train_n, y_train_n,
    epochs=250,
    batch_size=5,
    validation_data=(X_val_n, y_val_n),
    verbose=0
)

print("Final train loss (Adam):", history_adam.history['loss'][-1])
print("Final val   loss (Adam):", history_adam.history['val_loss'][-1])


last_5_days = temps[-window_size:]  
inp = last_5_days.reshape(1, window_size, 1)


inp_n = (inp - x_mean) / x_std

pred_n_sgd  = model_sgd.predict(inp_n,  verbose=0)[0, 0]
pred_n_adam = model_adam.predict(inp_n, verbose=0)[0, 0]


pred_sgd  = pred_n_sgd  * y_std + y_mean
pred_adam = pred_n_adam * y_std + y_mean

print("\nLast 5 days (°C):", np.round(last_5_days, 2))
print(f"Predicted next temp (SGD) : {pred_sgd:.2f} °C")
print(f"Predicted next temp (Adam): {pred_adam:.2f} °C")



Number of samples: 6
Final train loss (SGD): 3.397282455352979e-14
Final val   loss (SGD): 2.3512301445007324
Final train loss (Adam): 8.315570454442422e-13
Final val   loss (Adam): 2.347770929336548

Last 5 days (°C): [84. 90. 23. 84. 89.]
Predicted next temp (SGD) : 104.05 °C
Predicted next temp (Adam): 84.54 °C
